In [ ]:
import pandas as pd
import numpy as np
import pymssql
import os

In [ ]:
server = " "
user = " "
password = " "
db_name = " "

conn = pymssql.connect(server, user, password, db_name)

In [ ]:
base_query = """

             """

In [ ]:
base = pd.read_sql(base_query,conn)        
base.head()
df=base
df.head(5)

In [ ]:
#Example for pivoting the dataframe
df_final_pivot = pd.pivot_table(df[['Created_By', 'Ticket_Status', 'Ticket_No']],
                                            index=['Created_By'],
                                            columns=['Ticket_Status'], aggfunc=['count'], fill_value=0, ## dropna=True,
                                            values=['Ticket_No'])
df_final_pivot = pd.DataFrame(df_final_pivot.to_records())

In [ ]:
df_final_pivot.head()

In [ ]:
df_final_pivot.columns

In [ ]:
#Renaming the column names and deriving new names using dictonary
df_final_pivot=df_final_pivot.rename(columns={"('count', 'Ticket_No', 'COMPLETED')" :"Completed_Tickets",
                                              "('count', 'Ticket_No', 'Completed')":"Completed_Tickets",
                                              "('count', 'Ticket_No', 'Pending')" : "Pending_Tickets",
                                              "('count', 'Ticket_No', 'pending')" : "Pending_Tickets"
                                             })
df_final_pivot.head()

In [ ]:
#Adding new column and sum all the columns
df_final_pivot['Total']=df_final_pivot.sum(axis=1)
df_final_pivot.head()

In [ ]:
#Sum of the columns with the same name
df_final_pivot['Tickets_Completed_count']=df_final_pivot['Completed_Tickets'].sum(axis=1)
df_final_pivot['Tickets_Pending_count']=df_final_pivot['Pending_Tickets'].sum(axis=1)
df_final_pivot.head(25)

In [ ]:
#Calculating percentage and derving a new column
#df_final_pivot['FTR']=df_final_pivot['Total']/df_final_pivot["('count', 'Ticket_No', 'Completed')"]*100
df_final_pivot['FTR']=df_final_pivot[('Tickets_Completed_count')]/df_final_pivot['Total']*100
#df_final_pivot['FTR_round']=round(df_final_pivot['FTR'])
df_final_pivot['FTR_round']=np.round(df_final_pivot['FTR'],0)
df_final_pivot.head()

In [ ]:
#Ranking the percentage column
#df_final_pivot['Emp_Rank']=df_final_pivot.groupby('Created_By')['FTR_round'].rank("dense", ascending=False)
df_final_pivot['Emp_Rank']=df_final_pivot['FTR_round'].rank()
df_final_pivot.head()

In [ ]:
#Exporting the data in csv file format
df_final_pivot.to_csv('Current_month-hd.csv')